# Peak guesser
provided a peak energy and experiment, will suggest possible nuclides

In [11]:
 %matplotlib inline
# imports
import basic_data_reader as bdr
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import fispact_printlib_reader as fpr
import fispact_analysis as fa
import fispact_output_reader

# get gamma line data
linedata = fpr.read_fispact_printlib("Data/decay_lines.out")
linedata = fpr.particle_filter(linedata, 'gamma')
linedata["energy_kev"] = linedata["energy_ev"]/1000

In [16]:
# input data
peak_energy = 135 # energy in KeV of peak to be identified
p_energy = 30.0 # proton beam energy in MeV
irrad_time = 3600 # irradiation time in seconds  3600, 900 and 300 are simulated in fispact
material = "Ta"  # sample material

tol = 3 # energy tolerance on the peak energy - result provides all possible peaks within +- tol


In [17]:
# get fispact simulation output for the start of the count
ef = str(p_energy) + "MeV/"
fn1 = material + "_" + str(p_energy)
if irrad_time == 3600:
    tme_str = "1hr/"
elif irrad_time == 900:
    tme_str = "15min/"
else:
    tme_str = "5min/"
path = "simulations/fispact/"+ tme_str + material + "/TENDL/" + fn1 + ".out"
fout = fispact_output_reader.read_fis_out(path)

# reduce to gamma emitting nuclides with more than 1bq/g,
inv = fout.timestep_data[2].inventory  # need to make it link to cooling time - current set for 10mins
inv = inv[inv["act"] > 1000]   # simulations are for 1kg
inv = fa.filter_emits_gamma(inv)
nucs = inv["nuclide"].unique()

# search for possible lines
possibles = []
max_e = peak_energy + tol
min_e = peak_energy - tol

for i, nuc in enumerate(inv["nuclide"]):
    nuc_lines = linedata[linedata["nuclide"] == nuc]
    nuc_lines = nuc_lines.drop_duplicates(['energy_kev'])
    for energy in nuc_lines["energy_kev"]:
         if energy > min_e and energy < max_e:
                possibles.append((nuc, energy))
                
print(possibles)

[('Ta177', 136.725), ('W179', 133.9), ('W181', 136.26)]
